In [1]:
import os
import pandas as pd
import re
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('6502ops.csv', na_values=['nil'])
df.head(5)

,opcode,mnemonic,addressing mode,bytes,cycles,flags
0,0x69,ADC,IMM,2,2,CZidbVN
1,0x65,ADC,ZP,2,3,CZidbVN
2,0x75,ADC,ZPX,2,4,CZidbVN
3,0x6d,ADC,ABS,3,4,CZidbVN
4,0x7d,ADC,ABSX,3,4,CZidbVN


In [3]:
addressing_mode = df['addressing mode'].unique()
addressing_mode

array(['IMM', 'ZP', 'ZPX', 'ABS', 'ABSX', 'ABSY', 'INDX', 'INDY', 'ACC',
       'REL', 'IMP', 'IND', 'ZPY'], dtype=object)

In [4]:
opcodes_array = pd.DataFrame([],columns =['opcode','addressing_mode','function_name','num_bytes','num_cycles'])

for i in range(256):
        opcodes_array.loc[i,'function_name'] = 'NOT_SUPPORTED'  

for index, row in df.iterrows():
    opcodes_array.loc[int(row['opcode'],16),'opcode'] = row['mnemonic']
    opcodes_array.loc[int(row['opcode'],16),'addressing_mode'] = row['addressing mode']
    opcodes_array.loc[int(row['opcode'],16),'function_name'] = row['mnemonic'] + '_' + row['addressing mode']
    opcodes_array.loc[int(row['opcode'],16),'num_bytes'] = row['bytes']
    opcodes_array.loc[int(row['opcode'],16),'num_cycles'] = row['cycles']

In [5]:
max(opcodes_array['num_bytes'])

3

In [6]:
for index, row in opcodes_array.iterrows():
    if row['function_name'] != 'NOT_SUPPORTED':
        print('uint8_t ' + row['function_name'] + '()')
        print('{')
        print('\t uint8_t target[2];')
        print('\t uint16_t srcdst;')
        
        print('\t printf("' + '0x{:02x}'.format(index) + ' ");')
        
        if row['num_bytes'] >1 :
            print('\t target[0] = _iob->readMemory(_pc++);')
            print('\t printf("%#04x ", target[0]);')
        if row['num_bytes'] > 2:
            print('\t target[1] = _iob->readMemory(_pc++);')
            print('\t printf("%#04x ", target[1]);')
            
        print('\t printf("\t");')
        
        if row['addressing_mode'] == 'IMM':
            print('\t srcdst = target[0];')
        elif row['addressing_mode'] == 'ZP':
            print('\t srcdst = _iob->readMemory(target[0]);')
        elif row['addressing_mode'] == 'ZPX':
            print('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);')
        elif row['addressing_mode'] == 'ZPY':
            print('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _y) & 0x00FF);')
        elif row['addressing_mode'] == 'ABS':
            print('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);')
        elif row['addressing_mode'] == 'ABSX':
            print('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;')
        elif row['addressing_mode'] == 'ABSY':
            print('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _y;')
        elif row['addressing_mode'] == 'IND':
            print('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);')
            print('\t target[0] = _iob->readMemory(srcdst);')
            print('\t target[1] = _iob->readMemory(srcdst+1);')
            print('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);')
        elif row['addressing_mode'] == 'INDX':
            print('\t srcdst = static_cast<uint16_t>(target[0]);')
            print('\t target[0] = _iob->readMemory(srcdst);')
            print('\t srcdst = _iob->readMemory(static_cast<uint16_t>(target[0] + _x) & 0x00FF);')
        elif row['addressing_mode'] == 'INDY':
            print('\t srcdst = static_cast<int16_t>(target[0]);')
            print('\t target[0] = _iob->readMemory(srcdst);')
            print('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _y) & 0x00FF);')
        if row['addressing_mode'] == 'ACC':
            print('\t srcdst = _a;')
        if row['addressing_mode'] == 'REL':
            print('\t srcdst = target[0];')
        
        if row['num_bytes'] == 1 :  
            print('\t printf("'+ row['opcode'] + ' %#04x", static_cast<uint8_t>(srcdst));')
        if row['num_bytes'] == 2 :
            print('\t printf("'+ row['opcode'] + ' %#06x", srcdst);')
       
        print('\t ' + '_generic_' + row['opcode'] + '(srcdst);')
        print('\t return ' + str(row['num_cycles']) + ';')
        print('}\n')
    
print('uint8_t NOT_SUPPORTED() {return 0;}')

uint8_t BRK_IMP()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0x00 ");
	 printf("	");
	 printf("BRK %#04x", static_cast<uint8_t>(srcdst));
	 _generic_BRK(srcdst);
	 return 7;
}

uint8_t ORA_INDX()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0x01 ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0]);
	 printf("	");
	 srcdst = static_cast<uint16_t>(target[0]);
	 target[0] = _iob->readMemory(srcdst);
	 srcdst = _iob->readMemory(static_cast<uint16_t>(target[0] + _x) & 0x00FF);
	 printf("ORA %#06x", srcdst);
	 _generic_ORA(srcdst);
	 return 6;
}

uint8_t NOP_ZP()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0x04 ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0]);
	 printf("	");
	 srcdst = _iob->readMemory(target[0]);
	 printf("NOP %#06x", srcdst);
	 _generic_NOP(srcdst);
	 return 3;
}

uint8_t ORA_ZP()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0x05 ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0

	 return 2;
}

uint8_t LDY_ABSX()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0xbc ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0]);
	 target[1] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[1]);
	 printf("	");
	 srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;
	 _generic_LDY(srcdst);
	 return 4;
}

uint8_t LDA_ABSX()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0xbd ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0]);
	 target[1] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[1]);
	 printf("	");
	 srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;
	 _generic_LDA(srcdst);
	 return 4;
}

uint8_t LDX_ABSY()
{
	 uint8_t target[2];
	 uint16_t srcdst;
	 printf("0xbe ");
	 target[0] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[0]);
	 target[1] = _iob->readMemory(_pc++);
	 printf("%#04x ", target[1]);
	 printf("	");
	 srcdst = static_

In [7]:
OP_ABS_X_NO_CHECK_PAGE = ['ASL', 'DEC', 'INC', 'LSR', 'ROL', 'ROR', 'STA']
f = open('opcodes.inl','w')
for index, row in opcodes_array.iterrows():
    if row['function_name'] != 'NOT_SUPPORTED':
        f.write('uint8_t ' + row['function_name'] + '()\n')
        f.write('{\n')
        if row['addressing_mode'] != 'IMP' :
            f.write('\t uint8_t target[2];\n')
            f.write('\t uint16_t srcdst;\n')
            f.write('\t bool needLoad = false;\n')
        
        f.write('\t printf("' + '0x{:02x}'.format(index) + ' ");\n')
        
        if row['num_bytes'] >1 :
            f.write('\t target[0] = _iob->readMemory(_pc++);\n')
            f.write('\t printf("%#04x ", (uint8_t)target[0]);\n')
        if row['num_bytes'] > 2:
            f.write('\t target[1] = _iob->readMemory(_pc++);\n')
            f.write('\t printf("%#04x ", (uint8_t)target[1]);\n')
            
        f.write('\t printf("\t");')
        
        if row['addressing_mode'] == 'IMM':
            f.write('\t srcdst = target[0];\n')
        elif row['addressing_mode'] == 'ZP':
       #     f.write('\t srcdst = _iob->readMemory(target[0]);')
            f.write('\t srcdst = target[0];\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPX':
       #     f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);')
            f.write('\t srcdst = static_cast<uint8_t>(target[0] + _x) & 0x00FF;\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPY':
       #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _y) & 0x00FF);')
            f.write('\t srcdst = (static_cast<uint8_t>(target[0] + _y) & 0x00FF);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ABS':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ABSX':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;\n')
            f.write('\t needLoad = true;\n')
            if row['opcode'] not in OP_ABS_X_NO_CHECK_PAGE:
                f.write('\t _checkPageCrossing(srcdst, srcdst -_x);')
        elif row['addressing_mode'] == 'ABSY':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _y;\n')
            if row['opcode'] != 'STA':
                f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'IND':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t // this is for JMP, which is bugged on the 6502, need to take it into account\n')
            f.write('\t target[0] = _iob->readMemory(srcdst);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst&0xFF00) | ((srcdst + 1)&0x00FF));\n')
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'INDX':
            f.write('\t srcdst = target[0];\n')
            f.write('\t target[0] = _iob->readMemory((srcdst + _x)%256);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst + _x + 1)%256);\n')
        #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);\n')
        # val = PEEK(PEEK((arg + X) % 256) + PEEK((arg + X + 1) % 256) * 256)
            f.write('\t srcdst = target[0] | (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'INDY':
            f.write('\t srcdst = static_cast<int16_t>(target[0]);\n')
            f.write('\t target[0] = _iob->readMemory(srcdst);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst + 1)%256);\n')
        # val = PEEK(PEEK(arg) + PEEK((arg + 1) % 256) * 256 + Y)
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + static_cast<uint16_t>(_y);\n')
            if row['opcode'] != 'STA':
                f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
            f.write('\t needLoad = true;\n') 
        if row['addressing_mode'] == 'ACC':
            f.write('\t srcdst = _a;\n')
        if row['addressing_mode'] == 'REL':
            f.write('\t srcdst = static_cast<int16_t>(static_cast<int8_t>(target[0]));\n')
    
        if row['num_bytes'] == 1 :  
            f.write('\t printf("\t\t'+ row['opcode'] + ' \t\t\t\t");\n')
        if row['num_bytes'] == 2 :  
            f.write('\t printf("\t'+ row['opcode'] + ' %#04x \t\t\t", (uint8_t)srcdst);\n')
        if row['num_bytes'] == 3 :
            f.write('\t printf("'+ row['opcode'] + ' %#06x \t\t\t", (uint16_t)srcdst);\n')
       
        if row['addressing_mode'] != 'IMP' and row['opcode'] != 'NOP' :
            f.write('\t ' + '_generic_' + row['opcode'] + '(srcdst, needLoad);\n')
        else:
            f.write('\t ' + '_generic_' + row['opcode'] + '();\n')    
        f.write('\t return ' + str(row['num_cycles']) + ';\n')
        f.write('}\n')
    
f.write('uint8_t NOT_SUPPORTED() {return 0;}\n')
f.close()

In [8]:
for index, row in opcodes_array.iterrows():
    if row['function_name'] != 'NOT_SUPPORTED':
        print('case '+ str(index) + ': _cycle_cnt += ' +row['function_name'] + '() ; break;')

print('default:')
print('\tif((opcode & 0x0C) == 0x0C) _pc+=2;')
print('\telse if((opcode & 0x0A) != 0x0A)_pc+=1;') 
print('\tprintf("%#04x \\t\\t\\tUNKNOWN \\t\\t\\t", opcode);NOT_SUPPORTED();') 

case 0: _cycle_cnt += BRK_IMP() ; break;
case 1: _cycle_cnt += ORA_INDX() ; break;
case 4: _cycle_cnt += NOP_ZP() ; break;
case 5: _cycle_cnt += ORA_ZP() ; break;
case 6: _cycle_cnt += ASL_ZP() ; break;
case 8: _cycle_cnt += PHP_IMP() ; break;
case 9: _cycle_cnt += ORA_IMM() ; break;
case 10: _cycle_cnt += ASL_ACC() ; break;
case 12: _cycle_cnt += NOP_ABS() ; break;
case 13: _cycle_cnt += ORA_ABS() ; break;
case 14: _cycle_cnt += ASL_ABS() ; break;
case 16: _cycle_cnt += BPL_REL() ; break;
case 17: _cycle_cnt += ORA_INDY() ; break;
case 21: _cycle_cnt += ORA_ZPX() ; break;
case 22: _cycle_cnt += ASL_ZPX() ; break;
case 24: _cycle_cnt += CLC_IMP() ; break;
case 25: _cycle_cnt += ORA_ABSY() ; break;
case 29: _cycle_cnt += ORA_ABSX() ; break;
case 30: _cycle_cnt += ASL_ABSX() ; break;
case 32: _cycle_cnt += JSR_ABS() ; break;
case 33: _cycle_cnt += AND_INDX() ; break;
case 36: _cycle_cnt += BIT_ZP() ; break;
case 37: _cycle_cnt += AND_ZP() ; break;
case 38: _cycle_cnt += ROL_ZP() ; break;

In [9]:
OP_ABS_X_NO_CHECK_PAGE = ['ASL', 'DEC', 'INC', 'LSR', 'ROL', 'ROR', 'STA']
f = open('opcodes.inl','w')
for index, row in opcodes_array.iterrows():
    if row['function_name'] != 'NOT_SUPPORTED':
        f.write('uint8_t ' + row['function_name'] + '()\n')
        f.write('{\n')
        if row['addressing_mode'] != 'IMP' and row['addressing_mode'] != 'INDY':
            f.write('\t uint8_t target[2];\n')
            f.write('\t uint16_t srcdst;\n')
            f.write('\t bool needLoad = false;\n')
        
   #      f.write('\t printf("' + '0x{:02x}'.format(index) + ' ");\n')
        
        if row['addressing_mode'] != 'INDY':
            if row['num_bytes'] >1 :
                f.write('\t target[0] = _iob->readMemory(_pc++);\n')
     #            f.write('\t printf("%#04x ", (uint8_t)target[0]);\n')
            if row['num_bytes'] > 2:
                f.write('\t target[1] = _iob->readMemory(_pc++);\n')
      #           f.write('\t printf("%#04x ", (uint8_t)target[1]);\n')
            
    #     f.write('\t printf("\t");')
        
        if row['addressing_mode'] == 'IMM':
            f.write('\t srcdst = target[0];\n')
        elif row['addressing_mode'] == 'ZP':
       #     f.write('\t srcdst = _iob->readMemory(target[0]);')
            f.write('\t srcdst = target[0];\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPX':
       #     f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);')
            f.write('\t srcdst = static_cast<uint8_t>(target[0] + _x) & 0x00FF;\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPY':
       #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _y) & 0x00FF);')
            f.write('\t srcdst = (static_cast<uint8_t>(target[0] + _y) & 0x00FF);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ABS':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ABSX':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;\n')
            f.write('\t needLoad = true;\n')
            if row['opcode'] not in OP_ABS_X_NO_CHECK_PAGE:
                f.write('\t _checkPageCrossing(srcdst, srcdst -_x);')
        elif row['addressing_mode'] == 'ABSY':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _y;\n')
            if row['opcode'] != 'STA':
                f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'IND':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t // this is for JMP, which is bugged on the 6502, need to take it into account\n')
            f.write('\t target[0] = _iob->readMemory(srcdst);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst&0xFF00) | ((srcdst + 1)&0x00FF));\n')
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'INDX':
            f.write('\t srcdst = target[0];\n')
            f.write('\t target[0] = _iob->readMemory((srcdst + _x)%256);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst + _x + 1)%256);\n')
        #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);\n')
        # val = PEEK(PEEK((arg + X) % 256) + PEEK((arg + X + 1) % 256) * 256)
            f.write('\t srcdst = target[0] | (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'INDY':
 #           f.write('\t srcdst = static_cast<int16_t>(target[0]);\n')
 #           f.write('\t target[0] = _iob->readMemory(srcdst);\n')
 #           f.write('\t target[1] = _iob->readMemory((srcdst + 1)%256);\n')
        # val = PEEK(PEEK(arg) + PEEK((arg + 1) % 256) * 256 + Y)
 #           f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + static_cast<uint16_t>(_y);\n')
            if row['opcode'] != 'STA':
                f.write('\t ' + '_generic_' + row['opcode'] + '(AddrINDY(true), true);\n')
            else:
                f.write('\t ' + '_generic_' + row['opcode'] + '(AddrINDY(false), true);\n')
       #         f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
       #     f.write('\t needLoad = true;\n') 
        if row['addressing_mode'] == 'ACC':
            f.write('\t srcdst = _a;\n')
        if row['addressing_mode'] == 'REL':
            f.write('\t srcdst = static_cast<int16_t>(static_cast<int8_t>(target[0]));\n')
    
#         if row['addressing_mode'] != 'INDY':
#             if row['num_bytes'] == 1 :  
 #                f.write('\t printf("\t\t'+ row['opcode'] + ' \t\t\t\t");\n')
 #            if row['num_bytes'] == 2 :  
 #                f.write('\t printf("\t'+ row['opcode'] + ' %#04x \t\t\t", (uint8_t)srcdst);\n')
 #            if row['num_bytes'] == 3 :
 #                f.write('\t printf("'+ row['opcode'] + ' %#06x \t\t\t", (uint16_t)srcdst);\n')
       
        if row['addressing_mode'] == 'INDY':
            f.write('\t')
        elif row['addressing_mode'] != 'IMP' and row['opcode'] != 'NOP' :
            f.write('\t ' + '_generic_' + row['opcode'] + '(srcdst, needLoad);\n')
        else:
            f.write('\t ' + '_generic_' + row['opcode'] + '();\n')    
        f.write('\t return ' + str(row['num_cycles']) + ';\n')
        f.write('}\n')
    
f.write('uint8_t NOT_SUPPORTED() {return 0;}\n')
f.close()

In [17]:
OP_ABS_X_NO_CHECK_PAGE = ['ASL', 'DEC', 'INC', 'LSR', 'ROL', 'ROR', 'STA']
OP_REFACT = ['INDX', 'INDY', 'ABS']
f = open('opcodes.inl','w')
for index, row in opcodes_array.iterrows():
    if row['function_name'] != 'NOT_SUPPORTED':
        f.write('uint8_t ' + row['function_name'] + '()\n')
        f.write('{\n')
        if row['addressing_mode'] != 'IMP' and not row['addressing_mode'] in OP_REFACT:
            f.write('\t uint8_t target[2];\n')
            f.write('\t uint16_t srcdst;\n')
            f.write('\t bool needLoad = false;\n')
        
        f.write('\t fprintf(_CPULOG, "' + '0x{:02x}'.format(index) + ' ");\n')
        
        if row['addressing_mode'] not in OP_REFACT:
            if row['num_bytes'] >1 :
                f.write('\t target[0] = _iob->readMemory(_pc++);\n')
                f.write('\t fprintf(_CPULOG, "%#04x ", (uint8_t)target[0]);\n')
            if row['num_bytes'] > 2:
                f.write('\t target[1] = _iob->readMemory(_pc++);\n')
                f.write('\t fprintf(_CPULOG, "%#04x \t", (uint8_t)target[1]);\n')
            
        f.write('\t fprintf(_CPULOG, "\t");')
        
        if row['addressing_mode'] == 'IMM':
            f.write('\t srcdst = target[0];\n')
        elif row['addressing_mode'] == 'ZP':
       #     f.write('\t srcdst = _iob->readMemory(target[0]);')
            f.write('\t srcdst = target[0];\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPX':
       #     f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);')
            f.write('\t srcdst = static_cast<uint8_t>(target[0] + _x) & 0x00FF;\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ZPY':
       #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _y) & 0x00FF);')
            f.write('\t srcdst = (static_cast<uint8_t>(target[0] + _y) & 0x00FF);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'ABS':
#             f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
#             f.write('\t needLoad = true;\n') 
            f.write('\t uint16_t addr  = AddrABS();\n')
            f.write('\t fprintf(_CPULOG, "\t'+ row['opcode'] + ' %#06x \t\t\t", addr);\n')
            if row['opcode'] != 'NOP':
                f.write('\t ' + '_generic_' + row['opcode'] + '(addr, true);\n')
            else:
                f.write('\t ' + '_generic_' + row['opcode'] + '();\n')
        elif row['addressing_mode'] == 'ABSX':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _x;\n')
            f.write('\t needLoad = true;\n')
            if row['opcode'] not in OP_ABS_X_NO_CHECK_PAGE:
                f.write('\t _checkPageCrossing(srcdst, srcdst -_x);')
        elif row['addressing_mode'] == 'ABSY':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + _y;\n')
            if row['opcode'] != 'STA':
                f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'IND':
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t // this is for JMP, which is bugged on the 6502, need to take it into account\n')
            f.write('\t target[0] = _iob->readMemory(srcdst);\n')
            f.write('\t target[1] = _iob->readMemory((srcdst&0xFF00) | ((srcdst + 1)&0x00FF));\n')
            f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8);\n')
            f.write('\t needLoad = true;\n') 
        elif row['addressing_mode'] == 'INDX':
#             f.write('\t srcdst = target[0];\n')
#             f.write('\t target[0] = _iob->readMemory((srcdst + _x)%256);\n')
#             f.write('\t target[1] = _iob->readMemory((srcdst + _x + 1)%256);\n')
#         #    f.write('\t srcdst = _iob->readMemory(static_cast<uint8_t>(target[0] + _x) & 0x00FF);\n')
#         # val = PEEK(PEEK((arg + X) % 256) + PEEK((arg + X + 1) % 256) * 256)
#             f.write('\t srcdst = target[0] | (static_cast<uint16_t>(target[1]) << 8);\n')
#             f.write('\t needLoad = true;\n') 
            f.write('\t uint16_t addr  = AddrINDX();\n')
            f.write('\t fprintf(_CPULOG, "\t'+ row['opcode'] + ' %#06x \t\t\t", addr);\n')
            f.write('\t ' + '_generic_' + row['opcode'] + '(addr, true);\n')
        elif row['addressing_mode'] == 'INDY':
 #           f.write('\t srcdst = static_cast<int16_t>(target[0]);\n')
 #           f.write('\t target[0] = _iob->readMemory(srcdst);\n')
 #           f.write('\t target[1] = _iob->readMemory((srcdst + 1)%256);\n')
        # val = PEEK(PEEK(arg) + PEEK((arg + 1) % 256) * 256 + Y)
 #           f.write('\t srcdst = static_cast<uint16_t>(target[0]) + (static_cast<uint16_t>(target[1]) << 8) + static_cast<uint16_t>(_y);\n')
            f.write('\t uint16_t addr;\n')
            if row['opcode'] != 'STA':
                f.write('\t addr = AddrINDY(true); \n')
            else:
                f.write('\t addr = AddrINDY(false); \n')
            f.write('\t fprintf(_CPULOG, "\t'+ row['opcode'] + ' %#06x \t\t\t", addr);\n')
            f.write('\t ' + '_generic_' + row['opcode'] + '(addr, true);\n')
       #         f.write('\t _checkPageCrossing(srcdst, srcdst -_y);')
       #     f.write('\t needLoad = true;\n') 
        if row['addressing_mode'] == 'ACC':
            f.write('\t srcdst = _a;\n')
        if row['addressing_mode'] == 'REL':
            f.write('\t srcdst = static_cast<int16_t>(static_cast<int8_t>(target[0]));\n')
    
        if row['addressing_mode'] not in OP_REFACT:
            if row['num_bytes'] == 1 :  
                 f.write('\t fprintf(_CPULOG, "\t\t'+ row['opcode'] + ' \t\t\t\t");\n')
            if row['num_bytes'] == 2 :  
                 f.write('\t fprintf(_CPULOG, "\t'+ row['opcode'] + ' %#04x \t\t\t", (uint8_t)srcdst);\n')
            if row['num_bytes'] == 3 :
                 f.write('\t fprintf(_CPULOG, "'+ row['opcode'] + ' %#06x \t\t\t", (uint16_t)srcdst);\n')
       
        if row['addressing_mode'] in OP_REFACT:
            f.write('\t')
        elif row['addressing_mode'] != 'IMP' and row['opcode'] != 'NOP' :
            f.write('\t ' + '_generic_' + row['opcode'] + '(srcdst, needLoad);\n')
        else:
            f.write('\t ' + '_generic_' + row['opcode'] + '();\n')    
        f.write('\t return ' + str(row['num_cycles']) + ';\n')
        f.write('}\n')
    
f.write('uint8_t NOT_SUPPORTED() {return 0;}\n')
f.close()